## Lab 09
## Disruptive Algorithms and Frameworks: DeepMind's AlphaZero and Acme

#### Professor - Jeremy Curuksu, PhD (jeremy.cur@nyu.edu)

#### Section Leader - Anudeep Tubati (at5373@nyu.edu)

<hr>
<br>

Today, we will look at one of the most elegant algorithms in RL and the framework its creators use for day-to-day research. Personally, AlphaZero amazes me every single time I think about it. 

We will use Google Colab for today's lab since it does not have any pygame dependencies.

In [ ]:
# lets install the Acme framework
!pip install git+https://github.com/deepmind/acme.git#egg=dm-acme[jax,tf,envs]

DEPRECATION: git+https://github.com/deepmind/acme.git#egg=dm-acme[jax,tf,envs] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepmind/acme.git to /tmp/pip-install-09rjwzmw/dm-acme_66d0aa0372e94e0fa395d3ff77238759
  Running command git clone --filter=blob:none --quiet https://github.com/deepmind/acme.git /tmp/pip-install-09rjwzmw/dm-acme_66d0aa0372e94e0fa395d3ff77238759
  Resolved https://github.com/deepmind/acme.git to commit d1e69c92000079b118b868ce9303ee6d39c4a0b6
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 4.6 MB/s eta 0:00:0

In [34]:
import numpy as np

import gym
from gym.wrappers import RecordVideo

import dm_env
import acme
from acme import wrappers
from acme import specs
from acme.tf import networks
from acme.tf import utils as tf2_utils
from acme.agents.tf import dqn
from acme.utils import loggers

import sonnet as snt

import tensorflow as tf
import tensorflow_probability as tfp

import logging
logging.getLogger().setLevel(logging.DEBUG)  # help with seeing ACME logs

from IPython.display import Video

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
def make_environment(level: str = "CartPole-v1",
                     record: bool = False) -> dm_env.Environment:
    environment = gym.make(level)
    if record:
        environment = RecordVideo(environment, "./video", )

    environment = wrappers.GymWrapper(environment)
    environment = wrappers.SinglePrecisionWrapper(environment)

    return environment

In [27]:
env = make_environment()
env_spec = acme.make_environment_spec(env)

network = snt.Sequential([
      snt.Flatten(),
      snt.nets.MLP([50, 50, env_spec.actions.num_values]),
  ])

agent = dqn.DQN(env_spec, network,
               epsilon=0.1,
               learning_rate=1e-3,
               max_replay_size=1_000,
               batch_size=32)

# logger = loggers.TerminalLogger(time_delta=0.5)
env_loop = acme.EnvironmentLoop(env, agent)
env_loop.run(num_episodes=300)

/usr/local/lib/python3.9/dist-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
INFO:root:[Environment Loop] Env Reset Duration Sec = 0.001 | Env Step Duration Sec = 0.000 | Episode Duration = 0.289 | Episode Length = 9 | Episode Return = 9.0 | Episodes = 1 | Select Action Duration Sec = 0.026 | Steps = 9 | Steps Per Second = 31.152
INFO:root:[Environment Loop] Env Reset Duration Sec = 0.000 | Env Step Duration Sec = 0.000 | Episode Duration = 0.037 | Episode Length = 10 | Ep

56337

In [31]:
env = make_environment(record=True)

timestep = env.reset()

while not timestep.last():
  action = agent.select_action(timestep.observation)
  timestep = env.step(action)

/usr/local/lib/python3.9/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment CartPole-v1 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:51: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:51: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [37]:
Video("/content/video/rl-video-episode-0.mp4", embed=True)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
